In [42]:
!pip install -q tensorflow

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [3]:
# Load and preprocess data
train_data = pd.read_csv('neumf_train_data.csv')
train_data['verified_purchase'] = np.where(train_data['verified_purchase'] == 'Y', 1, 0)

test_data = pd.read_csv('neumf_test_data.csv')
test_data['verified_purchase'] = np.where(test_data['verified_purchase'] == 'Y', 1, 0)

num_users = len(train_data.customer_id.unique()) + len(test_data.customer_id.unique())
num_items = len(train_data.product_id.unique()) + len(test_data.product_id.unique())

# Convert to one hot encoded vector
train_users = to_categorical(train_data['customer_id'], num_classes=num_users)
train_items = to_categorical(train_data['product_id'], num_classes=num_items)
test_users = to_categorical(test_data['customer_id'], num_classes=num_users)
test_items = to_categorical(test_data['product_id'], num_classes=num_items)

In [18]:
test_items

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [19]:
# Define NeuMF model architecture
embedding_dim = 32
num_layers = 3
hidden_dim = 64

#input layer
user_input = Input(shape=(num_users,))
user_embedding_mlp = Flatten()(Embedding(num_users, embedding_dim)(user_input))
user_embedding_mf = Flatten()(Embedding(num_users,embedding_dim)(user_input))

item_input = Input(shape=(num_items,))
item_embedding_mlp = Flatten()(Embedding(num_items, embedding_dim)(item_input))
item_embedding_mf = Flatten()(Embedding(num_items,embedding_dim)(item_input))

mlp_layers = Concatenate()([user_embedding_mlp, item_embedding_mlp])
for i in range(num_layers):
    mlp_layers = Dense(hidden_dim, activation='relu')(mlp_layers)
    mlp_layers = Dropout(0.5)(mlp_layers)

mf_layers = tf.keras.layers.Multiply()([user_embedding_mf, item_embedding_mf])
# mf_layers = Concatenate()([user_embedding_mf, item_embedding_mf])
# mf_layers = Dense(hidden_dim, activation='relu')(mf_layers)

rating_output = Concatenate()([mlp_layers, mf_layers])
rating_output = Dense(1, activation='sigmoid')(rating_output)

verified_purchase_output = Concatenate()([mlp_layers, mf_layers])
verified_purchase_output = Dense(1, activation='sigmoid')(verified_purchase_output)

ValueError: ignored

In [6]:
# Define NeuMF model
neumf_model = Model(inputs=[user_input, item_input], outputs=[rating_output, verified_purchase_output])
# neumf_model.load_weights('neumf_model_weights.h5')

neumf_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy']) 

In [11]:
# Train NeuMF model
neumf_model.fit(x=[train_users, train_items],
                y=[train_data['star_rating'], train_data['verified_purchase']],
                batch_size=64,
                epochs=5,
                validation_data=([test_users, test_items],
                                 [test_data['star_rating'], test_data['verified_purchase']]))

# neumf_model.save_weights('neumf_model_weights.h5')

Epoch 1/5
510/510 [==============================] - 22s 43ms/step - loss: 3.1016 - tf.math.multiply_loss: 3.1016 - dense_5_loss: 1.8594e-15 - tf.math.multiply_accuracy: 0.0393 - dense_5_accuracy: 1.0000 - val_loss: -42.6358 - val_tf.math.multiply_loss: -42.6358 - val_dense_5_loss: 0.0000e+00 - val_tf.math.multiply_accuracy: 0.1884 - val_dense_5_accuracy: 1.0000
Epoch 2/5
510/510 [==============================] - 22s 44ms/step - loss: 3.1012 - tf.math.multiply_loss: 3.1012 - dense_5_loss: 2.9057e-18 - tf.math.multiply_accuracy: 0.0394 - dense_5_accuracy: 1.0000 - val_loss: -42.6358 - val_tf.math.multiply_loss: -42.6358 - val_dense_5_loss: 0.0000e+00 - val_tf.math.multiply_accuracy: 0.1884 - val_dense_5_accuracy: 1.0000
Epoch 3/5
510/510 [==============================] - 22s 43ms/step - loss: 3.1092 - tf.math.multiply_loss: 3.1092 - dense_5_loss: 1.0022e-17 - tf.math.multiply_accuracy: 0.0394 - dense_5_accuracy: 1.0000 - val_loss: -42.6358 - val_tf.math.multiply_loss: -42.6358 - val_d

In [12]:
def recommend_items(user_id, items=10):
    item_ids = np.arange(items)
    user_ids = np.repeat(user_id, items)
    user_ids_onehot = to_categorical(user_ids, num_classes=num_users)
    item_ids_onehot = to_categorical(item_ids, num_classes=num_items)
    rating_predictions, verified_purchase_predictions = neumf_model.predict([user_ids_onehot, item_ids_onehot])
    item_ratings = list(zip(item_ids, rating_predictions.flatten(), verified_purchase_predictions.flatten()))
    # item_ratings = [(item_id, rating, verified_purchase) for item_id, rating, verified_purchase in item_ratings
    #                 if verified_purchase >= 0.5 and rating > 3.0]
    item_ratings = [(item_id, rating, verified_purchase) for item_id, rating, verified_purchase in item_ratings]
    item_ratings.sort(key=lambda x: x[1], reverse=True)
    recommended_items = [(item_id, rating, verified_purchase) for item_id, rating, verified_purchase in item_ratings[:num_items]]
    return recommended_items

In [51]:
neumf_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 8120)]       0           []                               
                                                                                                  
 input_26 (InputLayer)          [(None, 7427)]       0           []                               
                                                                                                  
 embedding_48 (Embedding)       (None, 8120, 32)     259840      ['input_25[0][0]']               
                                                                                                  
 embedding_50 (Embedding)       (None, 7427, 32)     237664      ['input_26[0][0]']               
                                                                                            

In [ ]:
res = recommend_items(user_id=train_data.customer_id[0])
print(res)